In [ ]:
import os
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
# fix bug for the correct use of Conv2D layers
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

from helpers import normalize_img
from model_setup import training_fit_loop
from data_loading import Data

### Normalisation Function

In [ ]:
def mask_to_categorical(image, mask):
    image, mask = normalize_img(image, mask)
    mask = tf.one_hot(tf.cast(mask, tf.int32), 10)
    mask = tf.cast(mask, tf.float32)
    return image, mask

### Defining the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    return model

### Training

In [ ]:
a = training_fit_loop(model=define_model(),
                     model_params=((32, 32, 32), SGD, tf.keras.losses.CategoricalCrossentropy(), ["accuracy"]),
                     data_step=1500,
                     n=30,
                     data_loading_params=("cifar10", 128, mask_to_categorical, False, False, "test"),
                     start_data=4500,
                     save_df=os.getcwd()[:-len("tests")] + "results_df/", 
                     verbose=1)

In [ ]:
import pandas as pd

results = pd.read_pickle(os.getcwd()[:-len("tests")] + "results_df/07_07_2022_-16_44_06_experiment_1")
results

In [ ]:
results["Exp function fit"][0]

In [ ]:
results["PL function fit"][0]

### Test without loop